In [1]:
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
chunk = 1 # Fraction of data to train on
model_name = 'models_trained'
redownload=False


Here we download pre-trained models and a validation dataset. The models have been trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [3]:
if redownload: 
    downloader = file_fetcher.downloader(using_notebook=True)
    # Validation data
    downloader.download_file('https://www.dropbox.com/s/440m6x07bjg6c0h/Tweets.zip?dl=1',"Tweets.zip")
    
    # Compressed model
    # Note that this model has only been trained on 5% of the training data
    # Update when better-trained model is available
    downloader.download_file("https://www.dropbox.com/s/i88eqlja56xncyx/model_test_05.zip?dl=1","models_trained.zip")
    
    # Extract all the contents of zip file in current directory
    with ZipFile(model_name + '.zip', 'r') as zipObj:
        zipObj.extractall()

In [4]:
# Load the validation data

test_data = pd.read_csv('Tweets.zip', header=0, names=['Index', 'Sentiment', 'Sentiment_confidence',
                                                                'Negative_reason', 'Negative_reason_confidence',
                                                                'Airline', 'Airline_sentiment_gold', 'Handle',
                                                                'Negative_reason_gold', 'Retweet_count', 'Text',
                                                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [5]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [6]:
# Executing this cell takes about 30s on a laptop

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_models(model_name)

BoW model loaded successfully
LSTM model loaded successfully
Pre-trained embedding model loaded successfully


We santiy check the models: 

In [7]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

In [8]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


We test the model on an airline customer feedback dataset.

In [9]:
Classifier.evaluate(test_data['Text'], test_data['Labels'])

print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(test_data['Labels'], predictions)))
sklearn.metrics.confusion_matrix(test_data['Labels'], predictions)

BoW: 0.799
LSTM: 0.685
Pre-trained embedding: 0.766
Test Accuracy:  0.785
Test MCC:  0.538


array([[6974, 2204],
       [ 273, 2090]], dtype=int64)

We have accuracy of just under 80%.

To improve our accuracy, we can refine the model on our airline data. The early stopping procedure (enabled by default to use 10% of the training data for validation) should prevent overfitting. We withold 20% for our own validation. 

In [10]:
trainX, testX, trainY, testY = sklearn.model_selection.train_test_split(test_data['Text'], test_data['Labels'])

In [11]:
Classifier.refine(trainX, trainY)
test_predictions = Classifier.predict(testX)


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 3s 432us/step - loss: 0.4139 - acc: 0.8205 - val_loss: 0.3048 - val_acc: 0.8868
Epoch 2/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.3585 - acc: 0.8565 - val_loss: 0.2704 - val_acc: 0.8926
Epoch 3/250
7789/7789 [==============================] - 3s 362us/step - loss: 0.3221 - acc: 0.8750 - val_loss: 0.2506 - val_acc: 0.8949
Epoch 4/250
7789/7789 [==============================] - 3s 356us/step - loss: 0.2947 - acc: 0.8869 - val_loss: 0.2704 - val_acc: 0.8961
Epoch 5/250
7789/7789 [==============================] - 3s 360us/step - loss: 0.2958 - acc: 0.8928 - val_loss: 0.2855 - val_acc: 0.8949
Epoch 6/250
7789/7789 [==============================] - 3s 361us/step - loss: 0.3091 - acc: 0.8905 - val_loss: 0.2879 - val_acc: 0.8949
Epoch 7/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.3106 - acc: 0.8904 - val_loss: 0.2879 - val_acc: 0

7789/7789 [==============================] - 3s 359us/step - loss: 0.2124 - acc: 0.9176 - val_loss: 0.2018 - val_acc: 0.9099
Epoch 61/250
7789/7789 [==============================] - 3s 363us/step - loss: 0.1983 - acc: 0.9217 - val_loss: 0.2018 - val_acc: 0.9099
Epoch 62/250
7789/7789 [==============================] - 3s 363us/step - loss: 0.2016 - acc: 0.9200 - val_loss: 0.2022 - val_acc: 0.9099
Epoch 63/250
7789/7789 [==============================] - 3s 356us/step - loss: 0.1984 - acc: 0.9207 - val_loss: 0.2028 - val_acc: 0.9099
Epoch 64/250
7789/7789 [==============================] - 3s 360us/step - loss: 0.2034 - acc: 0.9210 - val_loss: 0.2036 - val_acc: 0.9088
Epoch 65/250
7789/7789 [==============================] - 3s 360us/step - loss: 0.1967 - acc: 0.9226 - val_loss: 0.2044 - val_acc: 0.9088
Epoch 66/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.2022 - acc: 0.9225 - val_loss: 0.1930 - val_acc: 0.9088
Epoch 67/250
7789/7789 [=======================

7789/7789 [==============================] - 3s 370us/step - loss: 0.1950 - acc: 0.9262 - val_loss: 0.1801 - val_acc: 0.9203
Epoch 120/250
7789/7789 [==============================] - 3s 361us/step - loss: 0.1966 - acc: 0.9269 - val_loss: 0.1814 - val_acc: 0.9203
Epoch 121/250
7789/7789 [==============================] - 3s 361us/step - loss: 0.1935 - acc: 0.9273 - val_loss: 0.1822 - val_acc: 0.9192
Epoch 122/250
7789/7789 [==============================] - 3s 360us/step - loss: 0.1956 - acc: 0.9285 - val_loss: 0.1825 - val_acc: 0.9169
Epoch 123/250
7789/7789 [==============================] - 3s 358us/step - loss: 0.1983 - acc: 0.9248 - val_loss: 0.1825 - val_acc: 0.9134
Epoch 124/250
7789/7789 [==============================] - 3s 361us/step - loss: 0.1968 - acc: 0.9275 - val_loss: 0.1827 - val_acc: 0.9145
Epoch 125/250
7789/7789 [==============================] - 3s 356us/step - loss: 0.1917 - acc: 0.9280 - val_loss: 0.1831 - val_acc: 0.9122
Epoch 126/250
7789/7789 [================

Epoch 178/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.1817 - acc: 0.9300 - val_loss: 0.1802 - val_acc: 0.9134
Epoch 179/250
7789/7789 [==============================] - 3s 356us/step - loss: 0.1808 - acc: 0.9308 - val_loss: 0.1811 - val_acc: 0.9134
Epoch 180/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.1736 - acc: 0.9299 - val_loss: 0.1817 - val_acc: 0.9134
Epoch 181/250
7789/7789 [==============================] - 3s 362us/step - loss: 0.1763 - acc: 0.9338 - val_loss: 0.1812 - val_acc: 0.9134
Epoch 182/250
7789/7789 [==============================] - 3s 358us/step - loss: 0.1807 - acc: 0.9318 - val_loss: 0.1787 - val_acc: 0.9145
Epoch 183/250
7789/7789 [==============================] - 3s 362us/step - loss: 0.1766 - acc: 0.9321 - val_loss: 0.1769 - val_acc: 0.9157
Epoch 184/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.1737 - acc: 0.9326 - val_loss: 0.1756 - val_acc: 0.9157
Epoch 185/250
7789/7789 [==

Epoch 237/250
7789/7789 [==============================] - 3s 357us/step - loss: 0.1616 - acc: 0.9357 - val_loss: 0.1883 - val_acc: 0.9169
Epoch 238/250
7789/7789 [==============================] - 3s 358us/step - loss: 0.1625 - acc: 0.9350 - val_loss: 0.1888 - val_acc: 0.9169
Epoch 239/250
7789/7789 [==============================] - 3s 359us/step - loss: 0.1655 - acc: 0.9371 - val_loss: 0.1891 - val_acc: 0.9169
Epoch 00239: early stopping


C:\Users\strix\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7789 samples, validate on 866 samples
Epoch 1/250
7789/7789 [==============================] - 4s 553us/step - loss: 1.0945 - acc: 0.7776 - val_loss: 0.8025 - val_acc: 0.8245
Epoch 2/250
7789/7789 [==============================] - 4s 490us/step - loss: 0.9182 - acc: 0.8054 - val_loss: 0.6855 - val_acc: 0.8291
Epoch 3/250
7789/7789 [==============================] - 4s 485us/step - loss: 0.7618 - acc: 0.8273 - val_loss: 0.6482 - val_acc: 0.8487
Epoch 4/250
7789/7789 [==============================] - 4s 484us/step - loss: 0.6535 - acc: 0.8471 - val_loss: 0.6292 - val_acc: 0.8626
Epoch 5/250
7789/7789 [==============================] - 4s 480us/step - loss: 0.6256 - acc: 0.8531 - val_loss: 0.6275 - val_acc: 0.8661
Epoch 6/250
7789/7789 [==============================] - 4s 482us/step - loss: 0.5465 - acc: 0.8601 - val_loss: 0.6252 - val_acc: 0.8741
Epoch 7/250
7789/7789 [==============================] - 4s 485us/step - loss: 0.5208 - acc: 0.8696 - val_loss: 0.6256 - val_acc: 0

7789/7789 [==============================] - 4s 479us/step - loss: 0.1757 - acc: 0.9462 - val_loss: 0.3676 - val_acc: 0.9076
Epoch 61/250
7789/7789 [==============================] - 4s 477us/step - loss: 0.1666 - acc: 0.9480 - val_loss: 0.3849 - val_acc: 0.9088
Epoch 62/250
7789/7789 [==============================] - 4s 479us/step - loss: 0.1601 - acc: 0.9466 - val_loss: 0.3842 - val_acc: 0.9111
Epoch 63/250
7789/7789 [==============================] - 4s 478us/step - loss: 0.1671 - acc: 0.9472 - val_loss: 0.3835 - val_acc: 0.9111
Epoch 64/250
7789/7789 [==============================] - 4s 481us/step - loss: 0.1623 - acc: 0.9490 - val_loss: 0.3829 - val_acc: 0.9111
Epoch 65/250
7789/7789 [==============================] - 4s 476us/step - loss: 0.1586 - acc: 0.9506 - val_loss: 0.3823 - val_acc: 0.9122
Epoch 66/250
7789/7789 [==============================] - 4s 481us/step - loss: 0.1549 - acc: 0.9490 - val_loss: 0.3817 - val_acc: 0.9145
Epoch 67/250
7789/7789 [=======================

In [12]:
print('Test Accuracy:  {:.3f}'.format(sklearn.metrics.accuracy_score(testY, test_predictions)))
print('Test MCC:  {:.3f}'.format(sklearn.metrics.matthews_corrcoef(testY, test_predictions)))
sklearn.metrics.confusion_matrix(testY, test_predictions)

Test Accuracy:  0.908
Test MCC:  0.714


array([[2172,  141],
       [ 125,  448]], dtype=int64)

Now we have accuracies of over 90%! 

In [13]:
Classifier.evaluate(testX, testY)

BoW: 0.889
LSTM: 0.840
Pre-trained embedding: 0.879


{'ensembled': 0.9078309078309078,
 'BoW': 0.8894663894663895,
 'LSTM': 0.8395703395703396,
 'Glove': 0.8794178794178794}

In [14]:
Classifier.save_models(model_name)

BoW model saved
LSTM model saved
Pre-trained embedding model saved
